In [1]:
from collections import defaultdict
import itertools

import pandas as pd

from chord_eval.metric import get_distance
from chord_eval.data_types import ChordType
from chord_eval.constants import TRIAD_REDUCTION

# Making the data
## Create all_distances.csv

In [4]:
root1s = []
root2s = []
chord_type1s = []
chord_type2s = []
inversion1s = []
inversion2s = []
spss = []
mechanicals = {
    1: [],
    2: [],
    3: [],
}
tone_by_tones = {
    0: {
        0: [],
        1: [],
        2: [],
    },
    1: {
        0: [],
        1: [],
        2: [],
    },
    2: {
        0: [],
        1: [],
        2: [],
    },
}

for root1 in range(0, 12):
    print(f"Checking distances from root {root1}")
    for root2 in range(root1, 12):
        for chord_type1 in ChordType:
            for chord_type2 in ChordType:
                for inversion1 in range(3 if chord_type1 in TRIAD_REDUCTION.values() else 4):
                    for inversion2 in range(3 if chord_type2 in TRIAD_REDUCTION.values() else 4):
                        sps = get_distance("SPS", root1, root2, chord_type1, chord_type2, inversion1=inversion1, inversion2=inversion2)

                        for bass_weight in [1, 2, 3]:
                            mechanicals[bass_weight].append(
                                get_distance(
                                    "mechanical",
                                    root1,
                                    root2,
                                    chord_type1,
                                    chord_type2,
                                    inversion1=inversion1,
                                    inversion2=inversion2,
                                    bass_weight=bass_weight,
                                )
                            )

                        for root_bonus in [0, 1, 2]:
                            for bass_bonus in [0, 1, 2]:
                                tone_by_tones[root_bonus][bass_bonus].append(
                                    get_distance(
                                        "tone by tone",
                                        root1,
                                        root2,
                                        chord_type1,
                                        chord_type2,
                                        inversion1=inversion1,
                                        inversion2=inversion2,
                                        bass_bonus=bass_bonus,
                                        root_bonus=root_bonus,
                                    )
                                )

                        root1s.append(root1)
                        root2s.append(root2)
                        chord_type1s.append(chord_type1)
                        chord_type2s.append(chord_type2)
                        inversion1s.append(inversion1)
                        inversion2s.append(inversion2)
                        spss.append(sps)

Checking distances from root 0
Checking distances from root 1
Checking distances from root 2
Checking distances from root 3
Checking distances from root 4
Checking distances from root 5
Checking distances from root 6
Checking distances from root 7
Checking distances from root 8
Checking distances from root 9
Checking distances from root 10
Checking distances from root 11


In [6]:
data_dict = {
    "root1": root1s,
    "root2": root2s,
    "chord_type1": chord_type1s,
    "chord_type2": chord_type2s,
    "inversion1": inversion1s,
    "inversion2": inversion2s,
    "sps": spss,
}

for bass_weight in [1, 2, 3]:
    data_dict[f"mechanical_b{bass_weight}"] = mechanicals[bass_weight]

for root_bonus in [0, 1, 2]:
    for bass_bonus in [0, 1, 2]:
        data_dict[f"tone_by_tone_r{root_bonus}_b{bass_bonus}"] = tone_by_tones[root_bonus][bass_bonus]

data = pd.DataFrame(data_dict)
data.to_csv("all_distances.csv", index=False)

## Load all_distances.csv

In [8]:
data = pd.read_csv("all_distances.csv")

for chord_type in ChordType:
    for i in ["1", "2"]:
        data.loc[data[f"chord_type{i}"] == str(chord_type), [f"chord_type{i}"]] = chord_type

In [9]:
data

,root1,root2,chord_type1,chord_type2,inversion1,inversion2,sps,mechanical_b1,mechanical_b2,mechanical_b3,tone_by_tone_r0_b0,tone_by_tone_r0_b1,tone_by_tone_r0_b2,tone_by_tone_r1_b0,tone_by_tone_r1_b1,tone_by_tone_r1_b2,tone_by_tone_r2_b0,tone_by_tone_r2_b1,tone_by_tone_r2_b2
0,0,0,ChordType.MAJOR,ChordType.MAJOR,0,0,0.719135,0,0,0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
1,0,0,ChordType.MAJOR,ChordType.MAJOR,0,1,0.180520,4,8,12,0.0,0.25,0.400000,0.0,0.200000,0.333333,0.0,0.166667,0.285714
2,0,0,ChordType.MAJOR,ChordType.MAJOR,0,2,0.040469,5,10,15,0.0,0.25,0.400000,0.0,0.200000,0.333333,0.0,0.166667,0.285714
3,0,0,ChordType.MAJOR,ChordType.MAJOR,1,0,0.000000,4,8,12,0.0,0.25,0.400000,0.0,0.200000,0.333333,0.0,0.166667,0.285714
4,0,0,ChordType.MAJOR,ChordType.MAJOR,1,1,0.180520,0,0,0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151003,11,11,ChordType.AUG_MAJ7,ChordType.AUG_MAJ7,2,3,0.024775,3,6,9,0.0,0.20,0.333333,0.0,0.166667,0.285714,0.0,0.142857,0.250000
151004,11,11,ChordType.AUG_MAJ7,ChordType.AUG_MAJ7,3,0,0.088996,1,2,3,0.0,0.20,0.333333,0.0,0.166667,0.285714,0.0,0.142857,0.250000
151005,11,11,ChordType.AUG_MAJ7,ChordType.AUG_MAJ7,3,1,0.000000,5,10,15,0.0,0.20,0.333333,0.0,0.166667,0.285714,0.0,0.142857,0.250000
151006,11,11,ChordType.AUG_MAJ7,ChordType.AUG_MAJ7,3,2,0.024775,3,6,9,0.0,0.20,0.333333,0.0,0.166667,0.285714,0.0,0.142857,0.250000


# Plotting

Text(0.5, 1.0, 'tone_by_tone_r1_b1')

In [28]:
import matplotlib.pyplot as plt
import numpy as np
import itertools

triads = [ChordType.MAJOR, ChordType.MINOR, ChordType.DIMINISHED, ChordType.AUGMENTED]
maj_min = [ChordType.MAJOR, ChordType.MINOR]

c_maj_mask = (data["root1"] == 0) & (data["chord_type1"] == ChordType.MAJOR) & (data["inversion1"] == 0)
c_maj_data = data.loc[c_maj_mask]

metric = "tone_by_tone_r1_b1"

matrix = np.zeros((12, 4 * 3))

for root in range(12):
    for i, (inv, chord_type) in enumerate(itertools.product(range(3), triads)):
        matrix[(11 - root), i] = c_maj_data.loc[(c_maj_data["root2"] == root) & (c_maj_data["chord_type2"] == chord_type) & (c_maj_data["inversion2"] == inv), metric]


plt.imshow(matrix)
plt.colorbar()
plt.xlabel("Chord Type")
plt.ylabel("Root")
plt.xticks(ticks=range(12), labels=[f"{chord_type}:{inv}" for inv, chord_type in itertools.product(range(3), ["M", "m", "o", "+"])])
plt.yticks(ticks=range(12), labels=["B", "Bb", "A", "Ab", "G", "Gb", "F", "E", "Eb", "D", "Db", "C"])
plt.title(metric)

Text(0.5, 1.0, 'sps')

In [25]:
import matplotlib.pyplot as plt
import numpy as np

triads = [ChordType.MAJOR, ChordType.MINOR, ChordType.DIMINISHED, ChordType.AUGMENTED]
maj_min = [ChordType.MAJOR, ChordType.MINOR]

c_maj_mask = (data["root1"] == 0) & (data["chord_type1"] == ChordType.MAJOR) & (data["inversion1"] == 0)
c_maj_data = data.loc[c_maj_mask]

metric = "sps"

matrix = np.zeros((12, 4 * 3))

for root in range(12):
    for i, (inv, chord_type) in enumerate(itertools.product(range(3), triads)):
        matrix[(11 - root), i] = c_maj_data.loc[(c_maj_data["root2"] == root) & (c_maj_data["chord_type2"] == chord_type) & (c_maj_data["inversion2"] == inv), metric]


plt.imshow(matrix)
plt.colorbar()
plt.xlabel("Chord Type")
plt.ylabel("Root")
plt.xticks(ticks=range(12), labels=[f"{chord_type}:{inv}" for inv, chord_type in itertools.product(range(3), ["M", "m", "o", "+"])])
plt.yticks(ticks=range(12), labels=["B", "Bb", "A", "Ab", "G", "Gb", "F", "E", "Eb", "D", "Db", "C"])
plt.title(metric)

Text(0.5, 1.0, 'mechanical_b1')

In [26]:
import matplotlib.pyplot as plt
import numpy as np

triads = [ChordType.MAJOR, ChordType.MINOR, ChordType.DIMINISHED, ChordType.AUGMENTED]
maj_min = [ChordType.MAJOR, ChordType.MINOR]

c_maj_mask = (data["root1"] == 0) & (data["chord_type1"] == ChordType.MAJOR) & (data["inversion1"] == 0)
c_maj_data = data.loc[c_maj_mask]

metric = "mechanical_b1"

matrix = np.zeros((12, 4 * 3))

for root in range(12):
    for i, (inv, chord_type) in enumerate(itertools.product(range(3), triads)):
        matrix[(11 - root), i] = c_maj_data.loc[(c_maj_data["root2"] == root) & (c_maj_data["chord_type2"] == chord_type) & (c_maj_data["inversion2"] == inv), metric]


plt.imshow(matrix)
plt.colorbar()
plt.xlabel("Chord Type")
plt.ylabel("Root")
plt.xticks(ticks=range(12), labels=[f"{chord_type}:{inv}" for inv, chord_type in itertools.product(range(3), ["M", "m", "o", "+"])])
plt.yticks(ticks=range(12), labels=["B", "Bb", "A", "Ab", "G", "Gb", "F", "E", "Eb", "D", "Db", "C"])
plt.title(metric)

# Metric Normalization

In [7]:
metric_names = ["sps"]

for bass_weight in [1, 2, 3]:
    metric_names.append(f"voice_leading_b{bass_weight}")

for root_bonus in [0, 1, 2]:
    for bass_bonus in [0, 1, 2]:
        metric_names.append(f"tone_by_tone_r{root_bonus}_b{bass_bonus}")

means = {metric: data[metric].mean() for metric in metric_names}
stds = {metric: data[metric].std() for metric in metric_names}

for metric in metric_names:
    data[f"{metric}_norm"] = (data[metric] - means[metric]) / stds[metric]

In [60]:
common_chord_types = [
    ChordType.MAJOR,
    ChordType.MINOR,
    ChordType.MAJ_MIN7,
    ChordType.DIMINISHED,
    ChordType.DIM7,
    ChordType.MIN_MIN7,
]

small_data = data.loc[data["chord_type1"].isin(common_chord_types) & data["chord_type2"].isin(common_chord_types)]

In [61]:
means

{'sps': 0.5721972574238494,
 'voice_leading_b1': 6.519422812036448,
 'voice_leading_b2': 9.512111941089215,
 'voice_leading_b3': 12.50480107014198,
 'tone_by_tone_r1_b1': 0.6874602670057318,
 'tone_by_tone_r1_b2': 0.7329227590599281,
 'tone_by_tone_r1_b3': 0.7630006357276999,
 'tone_by_tone_r2_b1': 0.7196638588690972,
 'tone_by_tone_r2_b2': 0.7519892985800853,
 'tone_by_tone_r2_b3': 0.7749693488336334,
 'tone_by_tone_r3_b1': 0.7409779614322934,
 'tone_by_tone_r3_b2': 0.765552629189523,
 'tone_by_tone_r3_b3': 0.7839234629007636,
 'sps_norm': 2.331218802954294,
 'voice_leading_b1_norm': 2.5782099022682177,
 'voice_leading_b2_norm': 2.3190060864632596,
 'voice_leading_b3_norm': 2.1563047619840146,
 'tone_by_tone_r1_b1_norm': 2.808877891202838,
 'tone_by_tone_r1_b2_norm': 3.390498652374484,
 'tone_by_tone_r1_b3_norm': 3.682457618579475,
 'tone_by_tone_r2_b1_norm': 3.084791906277165,
 'tone_by_tone_r2_b2_norm': 3.5782246824653314,
 'tone_by_tone_r2_b3_norm': 3.8496142873850983,
 'tone_by_to

In [62]:
stds

{'sps': 0.24544982937625243,
 'voice_leading_b1': 2.528662544621926,
 'voice_leading_b2': 4.101805509095662,
 'voice_leading_b3': 5.799180751536796,
 'tone_by_tone_r1_b1': 0.24474551533860991,
 'tone_by_tone_r1_b2': 0.21616960636292495,
 'tone_by_tone_r1_b3': 0.20719875549383127,
 'tone_by_tone_r2_b1': 0.23329413481790462,
 'tone_by_tone_r2_b2': 0.2101570933392389,
 'tone_by_tone_r2_b3': 0.20131090830887352,
 'tone_by_tone_r3_b1': 0.23738318497116112,
 'tone_by_tone_r3_b2': 0.2155461514136206,
 'tone_by_tone_r3_b3': 0.20503596913246125,
 'sps_norm': 1.0000000000000062,
 'voice_leading_b1_norm': 1.000000000001558,
 'voice_leading_b2_norm': 1.000000000000022,
 'voice_leading_b3_norm': 1.0000000000001834,
 'tone_by_tone_r1_b1_norm': 1.0000000000010405,
 'tone_by_tone_r1_b2_norm': 0.9999999999992142,
 'tone_by_tone_r1_b3_norm': 1.0000000000001172,
 'tone_by_tone_r2_b1_norm': 0.9999999999998724,
 'tone_by_tone_r2_b2_norm': 1.000000000000654,
 'tone_by_tone_r2_b3_norm': 1.0000000000004134,
 

In [20]:
data

,root1,root2,chord_type1,chord_type2,inversion1,inversion2,sps,voice_leading_b1,voice_leading_b2,voice_leading_b3,tone_by_tone_r1_b1,tone_by_tone_r1_b2,tone_by_tone_r1_b3,tone_by_tone_r2_b1,tone_by_tone_r2_b2,tone_by_tone_r2_b3,tone_by_tone_r3_b1,tone_by_tone_r3_b2,tone_by_tone_r3_b3,sps_norm,voice_leading_b1_norm,voice_leading_b2_norm,voice_leading_b3_norm,tone_by_tone_r1_b1_norm,tone_by_tone_r1_b2_norm,tone_by_tone_r1_b3_norm,tone_by_tone_r2_b1_norm,tone_by_tone_r2_b2_norm,tone_by_tone_r2_b3_norm,tone_by_tone_r3_b1_norm,tone_by_tone_r3_b2_norm,tone_by_tone_r3_b3_norm
0,0,0,ChordType.MAJOR,ChordType.MAJOR,0,0,0.000000,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
1,0,0,ChordType.MAJOR,ChordType.MAJOR,0,1,0.180520,4.0,8.0,12.0,0.0,0.25,0.400000,0.0,0.200000,0.333333,0.0,0.166667,0.285714,0.735466,1.581864,1.950361,2.069258,0.0,1.156499,1.930514,0.0,0.951669,1.655814,0.0,0.773230,1.393484
2,0,0,ChordType.MAJOR,ChordType.MAJOR,0,2,0.040469,5.0,10.0,15.0,0.0,0.25,0.400000,0.0,0.200000,0.333333,0.0,0.166667,0.285714,0.164877,1.977330,2.437951,2.586572,0.0,1.156499,1.930514,0.0,0.951669,1.655814,0.0,0.773230,1.393484
3,0,0,ChordType.MAJOR,ChordType.MAJOR,1,0,0.180520,4.0,8.0,12.0,0.0,0.25,0.400000,0.0,0.200000,0.333333,0.0,0.166667,0.285714,0.735466,1.581864,1.950361,2.069258,0.0,1.156499,1.930514,0.0,0.951669,1.655814,0.0,0.773230,1.393484
4,0,0,ChordType.MAJOR,ChordType.MAJOR,1,1,0.000000,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151003,11,11,ChordType.AUG_MAJ7,ChordType.AUG_MAJ7,2,3,0.024775,3.0,6.0,9.0,0.0,0.20,0.333333,0.0,0.166667,0.285714,0.0,0.142857,0.250000,0.100937,1.186398,1.462770,1.551943,0.0,0.925199,1.608761,0.0,0.793058,1.419269,0.0,0.662768,1.219298
151004,11,11,ChordType.AUG_MAJ7,ChordType.AUG_MAJ7,3,0,0.088996,1.0,2.0,3.0,0.0,0.20,0.333333,0.0,0.166667,0.285714,0.0,0.142857,0.250000,0.362582,0.395466,0.487590,0.517314,0.0,0.925199,1.608761,0.0,0.793058,1.419269,0.0,0.662768,1.219298
151005,11,11,ChordType.AUG_MAJ7,ChordType.AUG_MAJ7,3,1,0.103985,5.0,10.0,15.0,0.0,0.20,0.333333,0.0,0.166667,0.285714,0.0,0.142857,0.250000,0.423649,1.977330,2.437951,2.586572,0.0,0.925199,1.608761,0.0,0.793058,1.419269,0.0,0.662768,1.219298
151006,11,11,ChordType.AUG_MAJ7,ChordType.AUG_MAJ7,3,2,0.024775,3.0,6.0,9.0,0.0,0.20,0.333333,0.0,0.166667,0.285714,0.0,0.142857,0.250000,0.100937,1.186398,1.462770,1.551943,0.0,0.925199,1.608761,0.0,0.793058,1.419269,0.0,0.662768,1.219298


In [113]:
sorted = defaultdict(dict)
sorted_small = defaultdict(dict)

for metric1, metric2 in itertools.product(metric_names, metric_names):
    sorted_small[metric1][metric2] = (
        small_data[f"{metric1}_norm"] - small_data[f"{metric2}_norm"]
    ).abs().sort_values(ascending=False)

    sorted[metric1][metric2] = (
        data[f"{metric1}_norm"] - data[f"{metric2}_norm"]
    ).abs().sort_values(ascending=False)

# SPS and Tone by Tone

In [114]:
small_data.loc[
    sorted_small["sps"]["tone_by_tone_r1_b1"].index
].loc[
    (data["root1"] == 0) & (data["sps_norm"] > data["tone_by_tone_r1_b1_norm"]),
    ["root1", "root2", "chord_type1", "chord_type2", "inversion1", "inversion2", "sps", "sps_norm", "tone_by_tone_r1_b1", "tone_by_tone_r1_b1_norm", "voice_leading_b1", "voice_leading_b1_norm"],
][:30]

,root1,root2,chord_type1,chord_type2,inversion1,inversion2,sps,sps_norm,tone_by_tone_r1_b1,tone_by_tone_r1_b1_norm,voice_leading_b1,voice_leading_b1_norm
22603,0,11,ChordType.DIM7,ChordType.MAJ_MIN7,2,3,0.488563,-0.340740,0.250000,-1.787409,4.0,-0.996346
22606,0,11,ChordType.DIM7,ChordType.MAJ_MIN7,3,2,0.478767,-0.380649,0.250000,-1.787409,4.0,-0.996346
22636,0,11,ChordType.DIM7,ChordType.MIN_MIN7,3,0,0.721522,0.608373,0.500000,-0.765940,4.0,-0.996346
5941,0,3,ChordType.MINOR,ChordType.MAJOR,0,1,0.539440,-0.133458,0.333333,-1.446919,7.0,0.190052
22594,0,11,ChordType.DIM7,ChordType.MAJ_MIN7,0,2,0.454723,-0.478607,0.250000,-1.787409,7.0,0.190052
22600,0,11,ChordType.DIM7,ChordType.MAJ_MIN7,2,0,0.451391,-0.492184,0.250000,-1.787409,6.0,-0.205414
5948,0,3,ChordType.MINOR,ChordType.MAJOR,2,2,0.532994,-0.159720,0.333333,-1.446919,5.0,-0.600880
6082,0,3,ChordType.DIMINISHED,ChordType.MINOR,0,1,0.529038,-0.175839,0.333333,-1.446919,8.0,0.585518
22624,0,11,ChordType.DIM7,ChordType.MIN_MIN7,0,0,0.692637,0.490689,0.500000,-0.765940,2.0,-1.787278
21566,0,11,ChordType.DIMINISHED,ChordType.MAJOR,2,0,0.523668,-0.197714,0.333333,-1.446919,6.0,-0.205414


In [116]:
small_data.loc[
    sorted_small["sps"]["tone_by_tone_r1_b1"].index
].loc[
    (data["root1"] == 0) & (data["sps_norm"] < data["tone_by_tone_r1_b1_norm"]),
    ["root1", "root2", "chord_type1", "chord_type2", "inversion1", "inversion2", "sps", "sps_norm", "tone_by_tone_r1_b1", "tone_by_tone_r1_b1_norm", "voice_leading_b1", "voice_leading_b1_norm"],
][:30]

,root1,root2,chord_type1,chord_type2,inversion1,inversion2,sps,sps_norm,tone_by_tone_r1_b1,tone_by_tone_r1_b1_norm,voice_leading_b1,voice_leading_b1_norm
2013,0,1,ChordType.MAJOR,ChordType.MIN_MIN7,1,1,0.295014,-1.129287,0.75,0.255530,3.0,-1.391812
2709,0,1,ChordType.MAJ_MIN7,ChordType.MAJ_MIN7,1,1,0.548891,-0.094953,1.00,1.276999,4.0,-0.996346
4693,0,2,ChordType.MAJ_MIN7,ChordType.DIM7,1,1,0.549349,-0.093089,1.00,1.276999,5.0,-0.600880
1989,0,1,ChordType.MAJOR,ChordType.MAJ_MIN7,1,1,0.549997,-0.090449,1.00,1.276999,4.0,-0.996346
3961,0,2,ChordType.MAJOR,ChordType.DIM7,1,1,0.550742,-0.087411,1.00,1.276999,5.0,-0.600880
2222,0,1,ChordType.DIMINISHED,ChordType.DIMINISHED,1,1,0.551086,-0.086011,1.00,1.276999,3.0,-1.391812
3285,0,1,ChordType.DIM7,ChordType.DIM7,1,1,0.551379,-0.084817,1.00,1.276999,4.0,-0.996346
12674,0,6,ChordType.MIN_MIN7,ChordType.MAJOR,0,2,0.303077,-1.096437,0.75,0.255530,4.0,-0.996346
12346,0,6,ChordType.MAJ_MIN7,ChordType.DIMINISHED,0,2,0.303574,-1.094412,0.75,0.255530,4.0,-0.996346
2289,0,1,ChordType.DIMINISHED,ChordType.DIM7,1,1,0.554860,-0.070636,1.00,1.276999,5.0,-0.600880


# SPS and Voice Leading

In [117]:
small_data.loc[
    sorted_small["sps"]["voice_leading_b1"].index
].loc[
    (data["root1"] == 0) & (data["sps_norm"] > data["voice_leading_b1_norm"]),
    ["root1", "root2", "chord_type1", "chord_type2", "inversion1", "inversion2", "sps", "sps_norm", "tone_by_tone_r1_b1", "tone_by_tone_r1_b1_norm", "voice_leading_b1", "voice_leading_b1_norm"],
][:30]

,root1,root2,chord_type1,chord_type2,inversion1,inversion2,sps,sps_norm,tone_by_tone_r1_b1,tone_by_tone_r1_b1_norm,voice_leading_b1,voice_leading_b1_norm
21437,0,11,ChordType.MINOR,ChordType.MINOR,0,0,0.955547,1.561826,1.000000,1.276999,3.0,-1.391812
21578,0,11,ChordType.DIMINISHED,ChordType.DIMINISHED,0,0,0.941346,1.503968,1.000000,1.276999,3.0,-1.391812
21296,0,11,ChordType.MAJOR,ChordType.MAJOR,0,0,0.940558,1.500757,1.000000,1.276999,3.0,-1.391812
15503,0,8,ChordType.MAJOR,ChordType.MINOR,2,0,0.931684,1.464604,1.000000,1.276999,3.0,-1.391812
21304,0,11,ChordType.MAJOR,ChordType.MAJOR,2,2,0.930674,1.460488,1.000000,1.276999,3.0,-1.391812
21586,0,11,ChordType.DIMINISHED,ChordType.DIMINISHED,2,2,0.928591,1.452002,1.000000,1.276999,3.0,-1.391812
7879,0,4,ChordType.MINOR,ChordType.MAJOR,1,0,0.928500,1.451633,1.000000,1.276999,3.0,-1.391812
13821,0,7,ChordType.DIMINISHED,ChordType.MAJOR,1,2,0.920400,1.418633,1.000000,1.276999,3.0,-1.391812
13822,0,7,ChordType.DIMINISHED,ChordType.MAJOR,2,0,0.918750,1.411907,1.000000,1.276999,3.0,-1.391812
8020,0,4,ChordType.DIMINISHED,ChordType.MINOR,1,0,0.916251,1.401726,1.000000,1.276999,3.0,-1.391812


In [118]:
small_data.loc[
    sorted_small["sps"]["voice_leading_b1"].index
].loc[
    (data["root1"] == 0) & (data["sps_norm"] < data["voice_leading_b1_norm"]),
    ["root1", "root2", "chord_type1", "chord_type2", "inversion1", "inversion2", "sps", "sps_norm", "tone_by_tone_r1_b1", "tone_by_tone_r1_b1_norm", "voice_leading_b1", "voice_leading_b1_norm"],
][:30]

,root1,root2,chord_type1,chord_type2,inversion1,inversion2,sps,sps_norm,tone_by_tone_r1_b1,tone_by_tone_r1_b1_norm,voice_leading_b1,voice_leading_b1_norm
11898,0,6,ChordType.DIMINISHED,ChordType.DIMINISHED,0,0,0.303394,-1.095147,0.333333,-1.446919,12.0,2.167382
11906,0,6,ChordType.DIMINISHED,ChordType.DIMINISHED,2,2,0.330539,-0.984552,0.333333,-1.446919,12.0,2.167382
11901,0,6,ChordType.DIMINISHED,ChordType.DIMINISHED,1,0,0.088465,-1.970799,0.333333,-1.446919,9.0,0.980984
7069,0,3,ChordType.DIM7,ChordType.DIMINISHED,1,2,0.091673,-1.957728,0.250000,-1.787409,9.0,0.980984
17775,0,9,ChordType.DIMINISHED,ChordType.DIM7,0,3,0.097012,-1.935978,0.250000,-1.787409,9.0,0.980984
350,0,0,ChordType.DIMINISHED,ChordType.DIM7,0,2,0.097012,-1.935978,0.250000,-1.787409,9.0,0.980984
11964,0,6,ChordType.DIMINISHED,ChordType.DIM7,0,0,0.097012,-1.935978,0.250000,-1.787409,9.0,0.980984
6157,0,3,ChordType.DIMINISHED,ChordType.DIM7,0,1,0.097012,-1.935978,0.250000,-1.787409,9.0,0.980984
1262,0,0,ChordType.DIM7,ChordType.DIMINISHED,2,0,0.097012,-1.935978,0.250000,-1.787409,9.0,0.980984
18687,0,9,ChordType.DIM7,ChordType.DIMINISHED,2,1,0.109700,-1.884285,0.250000,-1.787409,9.0,0.980984


# Tone by Tone vs Voice Leading

In [119]:
small_data.loc[
    sorted_small["tone_by_tone_r1_b1"]["voice_leading_b1"].index
].loc[
    (data["root1"] == 0) & (data["tone_by_tone_r1_b1_norm"] > data["voice_leading_b1_norm"]),
    ["root1", "root2", "chord_type1", "chord_type2", "inversion1", "inversion2", "sps", "sps_norm", "tone_by_tone_r1_b1", "tone_by_tone_r1_b1_norm", "voice_leading_b1", "voice_leading_b1_norm"],
][:30]

,root1,root2,chord_type1,chord_type2,inversion1,inversion2,sps,sps_norm,tone_by_tone_r1_b1,tone_by_tone_r1_b1_norm,voice_leading_b1,voice_leading_b1_norm
2081,0,1,ChordType.MINOR,ChordType.MINOR,1,1,0.572615,0.001702,1.0,1.276999,3.0,-1.391812
2085,0,1,ChordType.MINOR,ChordType.MINOR,2,2,0.686218,0.464538,1.0,1.276999,3.0,-1.391812
7883,0,4,ChordType.MINOR,ChordType.MAJOR,2,1,0.683421,0.453143,1.0,1.276999,3.0,-1.391812
7879,0,4,ChordType.MINOR,ChordType.MAJOR,1,0,0.928500,1.451633,1.0,1.276999,3.0,-1.391812
2222,0,1,ChordType.DIMINISHED,ChordType.DIMINISHED,1,1,0.551086,-0.086011,1.0,1.276999,3.0,-1.391812
2218,0,1,ChordType.DIMINISHED,ChordType.DIMINISHED,0,0,0.822386,1.019308,1.0,1.276999,3.0,-1.391812
7878,0,4,ChordType.MINOR,ChordType.MAJOR,0,2,0.862631,1.183271,1.0,1.276999,3.0,-1.391812
2077,0,1,ChordType.MINOR,ChordType.MINOR,0,0,0.874005,1.229609,1.0,1.276999,3.0,-1.391812
13822,0,7,ChordType.DIMINISHED,ChordType.MAJOR,2,0,0.918750,1.411907,1.0,1.276999,3.0,-1.391812
13821,0,7,ChordType.DIMINISHED,ChordType.MAJOR,1,2,0.920400,1.418633,1.0,1.276999,3.0,-1.391812


In [120]:
small_data.loc[
    sorted_small["tone_by_tone_r1_b1"]["voice_leading_b1"].index
].loc[
    (data["root1"] == 0) & (data["tone_by_tone_r1_b1_norm"] < data["voice_leading_b1_norm"]),
    ["root1", "root2", "chord_type1", "chord_type2", "inversion1", "inversion2", "sps", "sps_norm", "tone_by_tone_r1_b1", "tone_by_tone_r1_b1_norm", "voice_leading_b1", "voice_leading_b1_norm"],
][:30]

,root1,root2,chord_type1,chord_type2,inversion1,inversion2,sps,sps_norm,tone_by_tone_r1_b1,tone_by_tone_r1_b1_norm,voice_leading_b1,voice_leading_b1_norm
11898,0,6,ChordType.DIMINISHED,ChordType.DIMINISHED,0,0,0.303394,-1.095147,0.333333,-1.446919,12.0,2.167382
11906,0,6,ChordType.DIMINISHED,ChordType.DIMINISHED,2,2,0.330539,-0.984552,0.333333,-1.446919,12.0,2.167382
356,0,0,ChordType.DIMINISHED,ChordType.DIM7,2,0,0.115450,-1.860856,0.250000,-1.787409,9.0,0.980984
355,0,0,ChordType.DIMINISHED,ChordType.DIM7,1,3,0.190022,-1.557041,0.250000,-1.787409,9.0,0.980984
350,0,0,ChordType.DIMINISHED,ChordType.DIM7,0,2,0.097012,-1.935978,0.250000,-1.787409,9.0,0.980984
6167,0,3,ChordType.DIMINISHED,ChordType.DIM7,2,3,0.115450,-1.860856,0.250000,-1.787409,9.0,0.980984
17775,0,9,ChordType.DIMINISHED,ChordType.DIM7,0,3,0.097012,-1.935978,0.250000,-1.787409,9.0,0.980984
17776,0,9,ChordType.DIMINISHED,ChordType.DIM7,1,0,0.190022,-1.557041,0.250000,-1.787409,9.0,0.980984
17781,0,9,ChordType.DIMINISHED,ChordType.DIM7,2,1,0.115450,-1.860856,0.250000,-1.787409,9.0,0.980984
6157,0,3,ChordType.DIMINISHED,ChordType.DIM7,0,1,0.097012,-1.935978,0.250000,-1.787409,9.0,0.980984


In [69]:
data.loc[
    (data["root1"] == 0) &
    (data["root2"] == 4) &
    (data["chord_type1"] == ChordType.MINOR) &
    (data["chord_type2"] == ChordType.MAJOR) &
    (data["inversion1"] == 1) &
    (data["inversion2"] == 0)
]

,root1,root2,chord_type1,chord_type2,inversion1,inversion2,sps,voice_leading_b1,voice_leading_b2,voice_leading_b3,tone_by_tone_r1_b1,tone_by_tone_r1_b2,tone_by_tone_r1_b3,tone_by_tone_r2_b1,tone_by_tone_r2_b2,tone_by_tone_r2_b3,tone_by_tone_r3_b1,tone_by_tone_r3_b2,tone_by_tone_r3_b3,sps_norm,voice_leading_b1_norm,voice_leading_b2_norm,voice_leading_b3_norm,tone_by_tone_r1_b1_norm,tone_by_tone_r1_b2_norm,tone_by_tone_r1_b3_norm,tone_by_tone_r2_b1_norm,tone_by_tone_r2_b2_norm,tone_by_tone_r2_b3_norm,tone_by_tone_r3_b1_norm,tone_by_tone_r3_b2_norm,tone_by_tone_r3_b3_norm,sps_norm_norm,voice_leading_b1_norm_norm,voice_leading_b2_norm_norm,voice_leading_b3_norm_norm,tone_by_tone_r1_b1_norm_norm,tone_by_tone_r1_b2_norm_norm,tone_by_tone_r1_b3_norm_norm,tone_by_tone_r2_b1_norm_norm,tone_by_tone_r2_b2_norm_norm,tone_by_tone_r2_b3_norm_norm,tone_by_tone_r3_b1_norm_norm,tone_by_tone_r3_b2_norm_norm,tone_by_tone_r3_b3_norm_norm
7879,0,4,ChordType.MINOR,ChordType.MAJOR,1,0,0.9285,3.0,4.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.451633,-1.391812,-1.343826,-1.294114,1.276999,1.235499,1.143826,1.201642,1.180121,1.117826,1.091156,1.08769,1.053847,-0.879585,-3.970022,-3.662832,-3.450419,-1.531879,-2.155,-2.538631,-1.883149,-2.398104,-2.731788,-2.030287,-2.463998,-2.769499
